In [ ]:

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from langchain_openai import ChatOpenAI
from newsapi import NewsApiClient
import smtplib
from email.mime.text import MIMEText
from datetime import datetime,timedelta
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
import smtplib
from email.mime.text import MIMEText
from langchain_core.tools import tool
from typing import Annotated
from langchain_core.utils.function_calling import convert_to_openai_function
from typing_extensions import TypedDict
from langchain.agents import create_openai_functions_agent
from langgraph.graph.message import AnyMessage, add_messages
from langchain import hub
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import END, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentFinish
from langgraph.graph import Graph, END


In [ ]:
SCOPES = ["https://www.googleapis.com/auth/calendar"]
NOTES={}
newsapi = NewsApiClient(api_key='')

In [ ]:
%set_env OPENAI_API_KEY=
%set_env TAVILY_API_KEY=

In [ ]:
@tool
def get_calender_info():
  """This code will trigger the Google API and fetch the next 10 nearing events and displays the start time and the 
   event name """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.credentials.json
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "/Users/praveenreddy/GenAI/Praveen_Agent/credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    now = datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    print("Getting the upcoming 10 events")
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")
      return

    # Prints the start and name of the next 10 events
    for event in events:
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])

  except HttpError as error:
    print(f"An error occurred: {error}")



In [ ]:
@tool
def tablets_reminder():
    """
    This method will just return a simple dictionary which holds the values of tablet name 
    and when to take it (morning/afternoon/night)
    """
    return[{'paracetmol':'morning'},{'dart':'afternoon'},{'kalpal':'night'}]

In [ ]:
@tool
def get_news_headlines():
    """This method will return the news headlines in india by triggering NEWS API"""
    titles = []
    top_headlines = newsapi.get_top_headlines( #q='headlines',
                                        #   sources='bbc-news,the-verge',
                                          category='general',
                                          language='en',
                                          country='in')
    for i in top_headlines['articles']:
      titles.append(i['title'],"\n\n")
    return titles


In [ ]:
@tool
def send_email(subject, body, sender, recipients, password=""):
    """This method will send an email (using Gmail API) with Subject and Body"""
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ', '.join(recipients)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
       smtp_server.login("mailpraveenreddy.c@gmail.com", password)
       smtp_server.sendmail(sender, recipients, msg.as_string())
    return "Message sent!"



# send_email(subject, body, sender, recipients, password)

In [ ]:
def list_to_llm(context):
    """This function will call LLM and it will share the context with LLM and asks LLM to come up with somthiing and return that text"""
    llm = ChatOpenAI()
    res = llm.invoke(f"Your are my assistant and you can build up some text around the context i share . Context is {context}")
    return res

# list_to_llm("give my some high motivation quotes")

In [ ]:
tools = [send_email,get_news_headlines,tablets_reminder,get_calender_info,TavilySearchResults()]

In [ ]:
prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt
llm = ChatOpenAI()

In [ ]:
agent_runnable = create_openai_functions_agent(llm,tools,prompt)
agent = RunnablePassthrough.assign(agent_outcome=agent_runnable)

In [ ]:
def execute_tools(data):
    agent_action = data.pop('agent_outcome')
    print("agent_action",agent_action)
    tool_to_use = {t.name: t for t in tools}[agent_action.tool]
    observation = tool_to_use.invoke(agent_action.tool_input)
    data['intermediate_steps'].append((agent_action, observation))
    return data

In [ ]:
def should_continue(data):
    if isinstance(data['agent_outcome'],AgentFinish):
        return "exit"
    else:
        return "continue"

In [ ]:
graph = Graph()
graph.add_node("agent",agent)
graph.add_node("tools",execute_tools)
graph.set_entry_point("agent")
graph.add_conditional_edges("agent",should_continue, {"continue":"tools","exit":END})
graph.add_edge("tools","agent")
chain = graph.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except:
    # This requires some extra dependencies and is optional
    pass

In [366]:
res = chain.invoke({"input":""" What are the meeting i HAVE TODAY IN my calender ""","intermediate_steps": []})
if isinstance(res['agent_outcome'], AgentFinish):
    # Extract the 'return_values' JSON string
    return_values_str = res['agent_outcome'].return_values
    print(return_values_str['output'])

agent_action tool='get_calender_info' tool_input={} log='\nInvoking: `get_calender_info` with `{}`\n\n\n' message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{}', 'name': 'get_calender_info'}}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 266, 'total_tokens': 278}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-93e9ca02-7004-46f7-9431-2a0e424840c5-0')]
Getting the upcoming 10 events
2024-06-08T14:15:00+05:30 Meeting with Bad Ass Boss
2024-06-08T16:30:00+05:30 Meeting with Aarushi
2024-06-08T19:15:00+05:30 Party WITH team mates
I couldn't fetch the calendar information at the moment. Can you please try again later?


In [367]:
res = chain.invoke({"input":"""Give me some 3 highly motivational quotes as I am bit down today  ""","intermediate_steps": []})
if isinstance(res['agent_outcome'], AgentFinish):
    # Extract the 'return_values' JSON string
    return_values_str = res['agent_outcome'].return_values
    print(return_values_str['output'])

Sure, here are 3 motivational quotes to uplift your spirits:

1. "Believe in yourself and all that you are. Know that there is something inside you that is greater than any obstacle." - Christian D. Larson

2. "Success is not final, failure is not fatal: It is the courage to continue that counts." - Winston Churchill

3. "The only way to do great work is to love what you do." - Steve Jobs

I hope these quotes inspire and motivate you!


In [368]:
res = chain.invoke({"input":"""My village name is Munagala and give me some 2 liner on my village,
                    praising the village and the culture we have in villages ""","intermediate_steps": []})
if isinstance(res['agent_outcome'], AgentFinish):
    # Extract the 'return_values' JSON string
    return_values_str = res['agent_outcome'].return_values
    print(return_values_str['output'])

Munagala, a serene village nestled in nature's embrace,
Where traditions thrive and community bonds grace.


In [369]:
res = chain.invoke({"input":"""i want to send an email to friend , his email id is mailpraveenreddy.c@gmail.com, 
                    subject should be my wife is not at home, come over, and the body should be drinks are waiting for
                      you buddy, lets party tonight, be here by 6.30PM , see you !! ""","intermediate_steps": []})
if isinstance(res['agent_outcome'], AgentFinish):
    # Extract the 'return_values' JSON string
    return_values_str = res['agent_outcome'].return_values
    print(return_values_str['output'])

agent_action tool='send_email' tool_input={'body': 'drinks are waiting for you buddy, lets party tonight, be here by 6.30PM , see you !!', 'recipients': 'mailpraveenreddy.c@gmail.com', 'sender': 'your email id', 'subject': 'my wife is not at home, come over'} log="\nInvoking: `send_email` with `{'body': 'drinks are waiting for you buddy, lets party tonight, be here by 6.30PM , see you !!', 'recipients': 'mailpraveenreddy.c@gmail.com', 'sender': 'your email id', 'subject': 'my wife is not at home, come over'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"body":"drinks are waiting for you buddy, lets party tonight, be here by 6.30PM , see you !!","recipients":"mailpraveenreddy.c@gmail.com","sender":"your email id","subject":"my wife is not at home, come over"}', 'name': 'send_email'}}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 323, 'total_tokens': 391}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint'

In [371]:
res = chain.invoke({"input":"who won the elections in telangana , india  ?","intermediate_steps": []})
if isinstance(res['agent_outcome'], AgentFinish):
    # Extract the 'return_values' JSON string
    return_values_str = res['agent_outcome'].return_values
    print(return_values_str['output'])

agent_action tool='tavily_search_results_json' tool_input={'query': 'Telangana election results'} log="\nInvoking: `tavily_search_results_json` with `{'query': 'Telangana election results'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Telangana election results"}', 'name': 'tavily_search_results_json'}}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 266, 'total_tokens': 288}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-c9694462-e37a-43af-8569-415fdc0b410f-0')]
The elections in Telangana resulted in a draw between the Congress and BJP, with both national parties leading in 8 of the 17 Lok Sabha seats.
